# Deliverable 3: optimizing the Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE

application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE


application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Visualize the value counts of APPLICATION_TYPE
#  YOUR CODE GOES HERE

application_type_counts.plot.density()

<AxesSubplot:ylabel='Density'>

In [6]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE

replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE


classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Visualize the value counts of CLASSIFICATION
#  YOUR CODE GOES HERE

classification_value_counts.plot.density()

<AxesSubplot:ylabel='Density'>

In [9]:
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE

replace_class = list(classification_value_counts[classification_value_counts < 800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE

application_cat= application_df.dtypes[application_df.dtypes =="object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE

encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE

application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Attempt 1: Dropping Noisy Variables. 2 Hidden layers, 80 and 30 neurons, 100 epochs.

In [13]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [14]:
not_noisy_application_df= application_df.drop(columns= ["USE_CASE_Other","AFFILIATION_Other", 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y'])
not_noisy_application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

y = not_noisy_application_df["IS_SUCCESSFUL"].values
X = not_noisy_application_df.drop(["IS_SUCCESSFUL"],1).values


# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-15-41a5c5aaeb64>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = not_noisy_application_df.drop(["IS_SUCCESSFUL"],1).values


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Compile, Train and Evaluate the Model

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3200      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,661
Trainable params: 5,661
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [19]:
# Create a callback which saves the weights for every 5 epochs
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

In [20]:
# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5695 - accuracy: 0.7226
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7326
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5488 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7335
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5475 - accuracy: 0.7339
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5473 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5371 - accuracy: 0.7398
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5374 - accuracy: 0.7404
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5373 - accuracy: 0.7402
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5372 - accuracy: 0.7409
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5367 - accuracy: 0.7408
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5370 - accuracy: 0.7397
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5369 - accuracy: 0.7402
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5367 - accuracy: 0.7405
Epoch 89/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5368 - accuracy: 0.7406
Epoch 90/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5367 - accuracy: 

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5553 - accuracy: 0.7259 - 556ms/epoch - 2ms/step
Loss: 0.5553111433982849, Accuracy: 0.7259474992752075


## Attempt 2: 3 Hidden layers, 80, 40, 20 neurons, 100 epochs.

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn1 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE

nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#  YOUR CODE GOES HERE

nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
#  YOUR CODE GOES HERE

nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3200      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7,281
Trainable params: 7,281
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

# Create a callback which saves the weights for every 5 epochs
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

# Train the model
fit_model= nn1.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5682 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5550 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7327
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5503 - accuracy: 0.7338
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5484 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7354
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5459 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5643 - accuracy: 0.7261 - 290ms/epoch - 1ms/step
Loss: 0.5642629265785217, Accuracy: 0.726064145565033


## Attempt 3: 3 Hidden layers, 100, 50, 30 neurons, 500 epochs.

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#  YOUR CODE GOES HERE

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))


# Output layer
#  YOUR CODE GOES HERE

nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               4000      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 10,611
Trainable params: 10,611
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

# Create a callback which saves the weights for every 5 epochs
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

# Train the model
fit_model= nn2.fit(X_train_scaled, y_train, epochs=500, verbose=1, callbacks=[cp_callback])

Epoch 1/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5691 - accuracy: 0.7212
Epoch 2/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5551 - accuracy: 0.7297
Epoch 3/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7309
Epoch 4/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7324
Epoch 5/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5490 - accuracy: 0.7336
Epoch 6/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5481 - accuracy: 0.7336
Epoch 7/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7329
Epoch 8/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7358
Epoch 9/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7342
Epoch 10/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5318 - accuracy: 0.7427
Epoch 161/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5315 - accuracy: 0.7427
Epoch 162/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5322 - accuracy: 0.7425
Epoch 163/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7425
Epoch 164/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5316 - accuracy: 0.7413
Epoch 165/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7414
Epoch 166/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7421
Epoch 167/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5325 - accuracy: 0.7424
Epoch 168/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5321 - accuracy: 0.7420
Epoch 169/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5313 - a

804/804 [==============================] - 4s 4ms/step - loss: 0.5322 - accuracy: 0.7418
Epoch 319/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5325 - accuracy: 0.7423
Epoch 320/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5329 - accuracy: 0.7417
Epoch 321/500
804/804 [==============================] - 4s 5ms/step - loss: 0.5337 - accuracy: 0.7415
Epoch 322/500
804/804 [==============================] - 5s 6ms/step - loss: 0.5329 - accuracy: 0.7426
Epoch 323/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5331 - accuracy: 0.7421
Epoch 324/500
804/804 [==============================] - 4s 4ms/step - loss: 0.5330 - accuracy: 0.7428
Epoch 325/500
804/804 [==============================] - 4s 5ms/step - loss: 0.5330 - accuracy: 0.7421
Epoch 326/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5336 - accuracy: 0.7420
Epoch 327/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5331 - a

804/804 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7380
Epoch 477/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5338 - accuracy: 0.7392
Epoch 478/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5332 - accuracy: 0.7403
Epoch 479/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7407
Epoch 480/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5345 - accuracy: 0.7404
Epoch 481/500
804/804 [==============================] - 4s 5ms/step - loss: 0.5356 - accuracy: 0.7399
Epoch 482/500
804/804 [==============================] - 4s 4ms/step - loss: 0.5341 - accuracy: 0.7405
Epoch 483/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5333 - accuracy: 0.7408
Epoch 484/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7417
Epoch 485/500
804/804 [==============================] - 4s 4ms/step - loss: 0.5332 - a

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5640 - accuracy: 0.7254 - 316ms/epoch - 1ms/step
Loss: 0.5639608502388, Accuracy: 0.7253644466400146


In [28]:
# Export our model to HDF5 file
nn2.save("AlphabetSoupCharity_Optimization.h5")